In [78]:
# default_exp core

In [79]:
#export
import re
from typing import Dict

import requests
from tqdm import trange

from bs4 import BeautifulSoup as bs
from nbdev.showdoc import *

In [80]:
#exports
class bioRxivScraper():
    """A simple class for scraping articles and their metadata from bioRxiv."""
    def __init__(self):
        url_advanced_search_base = "https://www.biorxiv.org/search/%20jcode%3Abiorxiv"
        # F-formatted string for advanced search querys
        url_advanced_search_params = ("%20subject_collection_code%3A{0}%20limit_from%3A{1}-{2}-"
                                      "01%20limit_to%3A{1}-{3}-01%20numresults%3A100%20sort%3Arelevance"
                                      "-rank%20format_result%3Astandard")

        self.url_advanced_search = url_advanced_search_base + url_advanced_search_params
        self.url_direct_link_base = 'https://www.biorxiv.org/{0}'
        
    def by_subject_area(self, subject_area: str, start_year: int, end_year: int = None) -> Dict:
        """Returns a dictionary keyed by doi, that contains the data/metadata of all articles uploaded 
        between `start_year` and `end_year` for a given `subject_area`.
        
        Args:
            subject_area (str): 
        """
        end_year = start_year if end_year is None else end_year
        # Format the subject area string such that it can be encoded in a URL
        url_encoded_subject_area = '%20'.join(subject_area.split(' '))

        scraped_content = {}

        for year in trange(start_year, end_year + 1):
            for month in trange(1, 2):
                # Populate the fields with our current query and post it
                resp = requests.post(
                    self.url_advanced_search.format(url_encoded_subject_area, year, month, month + 1)
                )
                html = bs(resp.text)
                # Collect the articles in the result in a list
                articles = html.find_all('li', attrs={'class': 'search-result'})
                for article in articles:
                    # Pull the title, if it's empty then skip it
                    linked_title =  article.find('a', attrs={'class': 'highwire-cite-linked-title'})
                    content_id = linked_title['href'].strip()
                    doi = '/'.join(content_id.split('/')[-2:])[:-2]
                    title = linked_title.find('span', attrs={'class': 'highwire-cite-title'})
                    authors = article.find_all('span', attrs={'class': 'highwire-citation-author'})

                    if title is None:
                        continue
                    title = title.text.strip()

                    resp = requests.post(self.url_direct_link_base.format(content_id))
                    html = bs(resp.text)

                    abstract = html.find("p", attrs={"id": re.compile(r"p-\d+")})

                    if abstract is None:
                        continue
                    abstract = abstract.text.strip()

                    # Collect year / month / title information
                    # TODO (John): Can we scrape the published date, and add only
                    # one field here: "data_published"
                    scraped_content[doi] = {
                        'month': month, 
                        'year': year, 
                        'title': title, 
                        'abstract': abstract,
                        'authors': [author.text for author in authors],
                    }

        return scraped_content

You need to run this cell in order to convert cells marked for export to a script

In [81]:
from nbdev.export import *
notebook2script()

Converted 00_core.ipynb.
Converted index.ipynb.
